In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

I've done the 1st task (if I think up something I will modify the code..)

https://www.kaggle.com/yutotsubaki/water-quality-1st-task-to-train

#### I try 2nd task
https://www.kaggle.com/adityakadiwal/water-potability/tasks?taskId=4773

EDA for water potability

In [ ]:
water_quality = pd.read_csv('/kaggle/input/water-potability/water_potability.csv')
water_quality.head()

In [ ]:
water_quality.shape

We have to recheck the number of null

In [ ]:
water_quality.isnull().sum() / len(water_quality) * 100

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(10, 10))


columns = water_quality.columns

indices = product([0, 1, 2], [0, 1, 2])

for col, idx in zip(columns, indices):
    sns.histplot(x=col, data=water_quality, kde=True, ax=axes[idx[0], idx[1]], bins=20)

last time, I fill the null with mean, but this time I fill them with Median

In [ ]:
water_quality.ph = water_quality.ph.fillna(water_quality.ph.median())
water_quality.Sulfate = water_quality.Sulfate.fillna(water_quality.Sulfate.median())
water_quality.Trihalomethanes = water_quality.Trihalomethanes.fillna(water_quality.Trihalomethanes.median())

In [ ]:
water_quality.isnull().sum() / len(water_quality) * 100

#### This time We have to do EDA for water potability

In [ ]:
water_quality.describe()

In [ ]:
# firstly we use pairplot

sns.pairplot(water_quality, hue='Potability')

### We can't find out pair of values which have high correlation.

#### but some values look different from each Potability

In [ ]:
var = 'Solids'
sns.boxplot(x='Potability', y=var, data=water_quality)

In [ ]:
var = 'Hardness'
sns.boxplot(x='Potability', y=var, data=water_quality)

In [ ]:
var = 'Conductivity'
sns.boxplot(x='Potability', y=var, data=water_quality)

#### I think we cut the bottom of 0.5% and top of 0.5% because the distance from start to 25% quantile and from 75% quantile to last

In [ ]:
# set the num of 99.5% quantile of Solids
outfilters_of_Solids = water_quality.Solids.quantile(0.995).tolist()

water_quality_without_outfilters_of_Solids = water_quality.copy()
water_quality_without_outfilters_of_Solids = water_quality_without_outfilters_of_Solids.query('Solids <= @outfilters_of_Solids')

In [ ]:
var = 'Solids'
sns.boxplot(x='Potability', y=var, data=water_quality_without_outfilters_of_Solids)

In [ ]:
# set the num of 99.5% quantile of Hardness
outfilters_of_left = water_quality_without_outfilters_of_Solids.Hardness.quantile(0.005).tolist()
outfilters_of_right = water_quality_without_outfilters_of_Solids.Hardness.quantile(0.995).tolist()


water_quality_without_outfilters_of_Solids = water_quality_without_outfilters_of_Solids.query('@outfilters_of_left <= Hardness <= @outfilters_of_right')

In [ ]:
var = 'Hardness'
sns.boxplot(x='Potability', y=var, data=water_quality_without_outfilters_of_Solids)

In [ ]:
# set the num of 99.9% quantile of Conductivity
# we can't cut over 5% because we have to keep quality and quantity as possible
outfilters_of_Conductivity = water_quality_without_outfilters_of_Solids.Conductivity.quantile(0.999).tolist()

water_quality_without_outfilters = water_quality_without_outfilters_of_Solids.query('Conductivity <= @outfilters_of_Conductivity')

In [ ]:
var = 'Conductivity'
sns.boxplot(x='Potability', y=var, data=water_quality_without_outfilters)

In [ ]:
water_quality_without_outfilters.shape

In [ ]:
water_quality_without_outfilters.describe()

#### We have still outfilters in Solids
#### so We check whether we can continue to cut down those outfilters aspect of the amount of cut samples

In [ ]:
(water_quality.shape[0] - water_quality_without_outfilters.shape[0]) / water_quality.shape[0] * 100

well, we've cut over 1%...

#### We cut outfilters again from first and adjust the percentage..

In [ ]:
# set the num of 99.5% quantile of Solids
outfilters_of_Solids_left = water_quality.Solids.quantile(0.003).tolist()
outfilters_of_Solids_right = water_quality.Solids.quantile(0.997).tolist()

water_quality_without_outfilters_of_Solids = water_quality.copy()
water_quality_without_outfilters_of_Solids = water_quality_without_outfilters_of_Solids.query('@outfilters_of_Solids_left <= Solids <= @outfilters_of_Solids_right')

In [ ]:
var = 'Solids'
sns.boxplot(x='Potability', y=var, data=water_quality_without_outfilters_of_Solids)

In [ ]:
# set the num of 99.5% quantile of Hardness
outfilters_of_left = water_quality_without_outfilters_of_Solids.Hardness.quantile(0.003).tolist()
outfilters_of_right = water_quality_without_outfilters_of_Solids.Hardness.quantile(0.995).tolist()


water_quality_without_outfilters_of_Hardness = water_quality_without_outfilters_of_Solids.query('@outfilters_of_left <= Hardness <= @outfilters_of_right')

In [ ]:
var = 'Hardness'
sns.boxplot(x='Potability', y=var, data=water_quality_without_outfilters_of_Hardness)

In [ ]:
# set the num of 99.9% quantile of Conductivity
# we can't cut over 5% because we have to keep quality and quantity as possible
outfilters_of_Conductivity = water_quality_without_outfilters_of_Hardness.Conductivity.quantile(0.999).tolist()

water_quality_without_outfilters = water_quality_without_outfilters_of_Hardness.query('Conductivity <= @outfilters_of_Conductivity')

In [ ]:
var = 'Conductivity'
sns.boxplot(x='Potability', y=var, data=water_quality_without_outfilters)

#### recheck how many outfilters we've cut

In [ ]:
(water_quality.shape[0] - water_quality_without_outfilters.shape[0]) / water_quality.shape[0] * 100

In [ ]:
water_quality_without_outfilters.describe()

I think we barely save the quality and quantity??, haha

### So, We retry to predict this data!!

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
data = water_quality_without_outfilters.copy()

In [ ]:
scaler = StandardScaler()

In [ ]:
X = data.drop('Potability', axis=1).values
y = data.copy()['Potability']

X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y)

In [ ]:
estimators = [
   ('rf', RandomForestClassifier(n_estimators=100)),
   ('pipeline', Pipeline([
                        ('pca', PCA(n_components=2)),
                        ('lr', LogisticRegression(max_iter=500))
                    ])
   )
]



stack = StackingClassifier(estimators=estimators, 
                           final_estimator=LogisticRegression(max_iter=1000))

scores = cross_val_score(estimator=stack,
                        X=X_train, y=y_train,
                        cv=10, scoring='accuracy')

np.mean(scores)

In [ ]:
estimators = [
   ('rf', RandomForestClassifier(n_estimators=100)),
   ('pipeline', Pipeline([
                        ('pca', PCA(n_components=2)),
                        ('lr', LogisticRegression(max_iter=500))
                    ])
   )
]



stack = StackingClassifier(estimators=estimators, 
                           final_estimator=LogisticRegression(max_iter=1000))

scores = cross_val_score(estimator=stack,
                        X=X_train, y=y_train,
                        cv=10, scoring='roc_auc')

np.mean(scores)

#### Well, It's higer than my last stacking models
#### so, why don't we use adaboost??

In [ ]:
from sklearn.ensemble import AdaBoostClassifier


tree = DecisionTreeClassifier(max_depth=None, criterion='entropy')

ada = AdaBoostClassifier(base_estimator=tree,  n_estimators=1000, learning_rate=0.1)


scores = cross_val_score(estimator=ada,
                        X=X_train, y=y_train,
                        cv=10, scoring='roc_auc')

np.mean(scores)

#### of course, that's worse..

#### so, finally we predict the test data and check acc

In [ ]:
stack.fit(X_train, y_train)

y_pred = stack.predict(X_test)

accuracy_score(y_true=y_test, y_pred=y_pred)

### Well, still not be enough acc but it's more accurate than my last prediction (which scores 0.625...)

### Let's go back to the filling null part.

#### We know the 'ph' and 'Sulfate' null rates are more than 10%, so we rethink how to fill and what we should fill with!

#### I think Trihalomethanes' null can be filled with mean (or median).

#### so, we try to fill 'ph' and 'Sulfate' null with predictions!

In [ ]:
water_quality = pd.read_csv('/kaggle/input/water-potability/water_potability.csv')
water_quality.head()

In [ ]:
water_quality['Trihalomethanes'] = water_quality['Trihalomethanes'].fillna(water_quality['Trihalomethanes'].median())

In [ ]:
# check
water_quality['Trihalomethanes'].isnull().sum()

### first, we try to predict ph by columns without Potability and Sulfate
#### and we use Linear Regression first.

In [ ]:
X_for_null = water_quality.dropna().drop(['Potability', 'Sulfate', 'ph'], axis=1).values
y_ph = water_quality.dropna().ph.values

scaler = StandardScaler()
X_for_null_scaled = scaler.fit_transform(X_for_null)

X_for_null_train, X_for_null_test, y_ph_train, y_ph_test = train_test_split(X_for_null_scaled, y_ph, test_size=0.2)

In [ ]:
X_for_null = water_quality.dropna().drop(['Potability', 'Sulfate', 'ph'], axis=1).values
y_ph = water_quality.dropna().ph.values.reshape(-1, 1)

scaler = StandardScaler()
X_for_null_scaled = scaler.fit_transform(X_for_null)

scaler = StandardScaler()
y_ph = scaler.fit_transform(y_ph)

X_for_null_train, X_for_null_test, y_ph_train, y_ph_test = train_test_split(X_for_null_scaled, y_ph, test_size=0.2)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold


model = LinearRegression()

kfold = KFold(n_splits=5).split(X_for_null_train, y_ph_train)

scores = []
for train, test in kfold:
    model.fit(X_for_null_train[train], y_ph_train[train])
    
    score = model.score(X_for_null_train[test], y_ph_train[test])
    
    scores.append(score)
    

# mean the cross validation scores!
np.mean(scores)

#### What!! this is quit bad...
#### so, we can't predict enough..

### Second, we use the fillna's method 'ffill'

In [ ]:
water_quality_ffill = water_quality.copy()
water_quality_ffill = water_quality_ffill.sort_values('ph')
water_quality_ffill.ph = water_quality_ffill.ph.fillna(method='ffill')

water_quality_ffill = water_quality_ffill.sort_values('Sulfate')
water_quality_ffill.Sulfate = water_quality_ffill.Sulfate.fillna(method='ffill')

water_quality_ffill.isnull().sum()

#### let's cut the outfilters again and then predict!!!!

In [ ]:
# set the num of 99.5% quantile of Solids
outfilters_of_Solids_left = water_quality_ffill.Solids.quantile(0.003).tolist()
outfilters_of_Solids_right = water_quality_ffill.Solids.quantile(0.997).tolist()

water_quality_without_outfilters_of_Solids = water_quality_ffill.copy()
water_quality_without_outfilters_of_Solids = water_quality_without_outfilters_of_Solids.query('@outfilters_of_Solids_left <= Solids <= @outfilters_of_Solids_right')

In [ ]:
# set the num of 99.5% quantile of Hardness
outfilters_of_left = water_quality_without_outfilters_of_Solids.Hardness.quantile(0.003).tolist()
outfilters_of_right = water_quality_without_outfilters_of_Solids.Hardness.quantile(0.995).tolist()


water_quality_without_outfilters_of_Hardness = water_quality_without_outfilters_of_Solids.query('@outfilters_of_left <= Hardness <= @outfilters_of_right')

In [ ]:
# set the num of 99.9% quantile of Conductivity
# we can't cut over 5% because we have to keep quality and quantity as possible
outfilters_of_Conductivity = water_quality_without_outfilters_of_Hardness.Conductivity.quantile(0.999).tolist()

water_quality_without_outfilters = water_quality_without_outfilters_of_Hardness.query('Conductivity <= @outfilters_of_Conductivity')

In [ ]:
scaler = StandardScaler()

X = water_quality_without_outfilters.drop('Potability', axis=1).values
y = water_quality_without_outfilters['Potability']

X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y)

In [ ]:
estimators = [
   ('rf', RandomForestClassifier(n_estimators=100)), 
   ('tree', DecisionTreeClassifier(max_depth=1)), 
   ('pipeline', Pipeline([
                        ('pca', PCA(n_components=2)),
                        ('lr', LogisticRegression(max_iter=500))
                    ])
   )
]



stack = StackingClassifier(estimators=estimators, 
                           final_estimator=LogisticRegression(max_iter=1000))

scores = cross_val_score(estimator=stack,
                        X=X_train, y=y_train,
                        cv=10, scoring='accuracy')

np.mean(scores)

### we try to tune the best param to use RandimizeSearch

In [ ]:
for key in stack.get_params().keys():
    print(key)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

estimators = [
   ('rf', RandomForestClassifier()), 
   ('tree', DecisionTreeClassifier()), 
   ('pipeline', Pipeline([
                        ('pca', PCA(n_components=2)),
                        ('lr', LogisticRegression(max_iter=500))
                    ])
   )
]



stack = StackingClassifier(estimators=estimators, 
                           final_estimator=LogisticRegression(max_iter=1000))

scores = cross_val_score(estimator=stack,
                        X=X_train, y=y_train,
                        cv=10, scoring='accuracy')

np.mean(scores)


param_dist = {'pipeline__lr__C': np.arange(0.01, 10), 
              'tree__max_depth':[1, 2, 3, None], 
              'tree__criterion': ['gini', 'entropy'], 
              'rf__max_depth':[1, 2, 3, None], 
              'final_estimator__C': np.arange(0.01, 10),
              'tree__max_features': range(1, 5)
             }


rand_search = RandomizedSearchCV(estimator=stack, param_distributions=param_dist, cv=5)
rand_search.fit(X_train, y_train)

rand_search.best_score_

#### still not be enough but we have to check the best param

In [ ]:
rand_search.best_params_

#### all right..

#### for the moment, we predict test data!

In [ ]:
rand_search.score(X_test,y_test)

#### so, this score got a little worse, so we try to change the scale instead of standardscaler
#### we replace the minmax

In [ ]:
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()

X = water_quality_without_outfilters.drop('Potability', axis=1).values
y = water_quality_without_outfilters['Potability']

X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y)

# we use the previous best param
estimators = [
   ('rf', RandomForestClassifier(max_depth=None)), 
   ('tree', DecisionTreeClassifier(max_features=2, max_depth=1, criterion='entropy')), 
   ('pipeline', Pipeline([
                        ('pca', PCA(n_components=2)),
                        ('lr', LogisticRegression(max_iter=500, C=9))
                    ])
   )
]

stack = StackingClassifier(estimators=estimators, 
                           final_estimator=LogisticRegression(max_iter=1000, C=3))

scores = cross_val_score(estimator=stack,
                        X=X_train, y=y_train,
                        cv=10, scoring='accuracy')

np.mean(scores)

#### It looks nearly above stacking model

In [ ]:
from sklearn.model_selection import StratifiedKFold

stack.fit(X_train, y_train)

y_pred = stack.predict(X_test)
accuracy_score(y_true=y_test, y_pred=y_pred)

### Conclusion for the moment.

#### I think we have to use deep learning if we want to get more accuracy
#### and this time, we had better use median to fill null and cut some outfilters!